In [3]:
import torch
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [4]:
# 假设有两个句子
batch_size = 2
# 每个句子长度为2~5
src_len = T.randint(2, 5, (batch_size, ))
tgt_len = T.randint(2, 5, (batch_size, ))
print(src_len)
print(tgt_len)
# 方便研究，我们写死
src_len = T.Tensor([2, 4]).to(T.int32)
tgt_len = T.Tensor([4, 3]).to(T.int32)

tensor([3, 3])
tensor([2, 2])


In [11]:
valid_encoder_pos = [torch.ones(L) for L in src_len]
# padding至max句子长度
valid_encoder_pos = list(map(lambda x: F.pad(x, (0, max(src_len) - len(x))), valid_encoder_pos))
# 扩1维
valid_encoder_pos = list(map(lambda x: T.unsqueeze(x, 0), valid_encoder_pos))
# 拼接
valid_encoder_pos = T.cat(valid_encoder_pos, 0)
# 继续扩维 -> [2,4,1]
valid_encoder_pos = T.unsqueeze(valid_encoder_pos, 2)
print(valid_encoder_pos.shape, "# valid_encoder_pos size")
# bmm：带批的矩阵相乘 [2,4,1] * [2,1,4]
valid_encoder_pos_matrix = torch.bmm(valid_encoder_pos, valid_encoder_pos.transpose(1, 2))
print(valid_encoder_pos_matrix.shape, "# valid_encoder_pos_matrix size")
print(valid_encoder_pos_matrix, "# 4*4，valid_encoder_pos_matrix 第一行表示第一个单词对其他单词的有效性")

invalid_encoder_pos_matrix = 1-valid_encoder_pos_matrix # 取反
print(invalid_encoder_pos_matrix, "# invalid_encoder_pos_matrix 0表示有效位置，1表示无效的位置")

mask_encoder_self_attention = invalid_encoder_pos_matrix.to(torch.bool)
print(mask_encoder_self_attention, "# mask_encoder_self_attention True的地方需要mask")

torch.Size([2, 4, 1]) # valid_encoder_pos size
torch.Size([2, 4, 4]) # valid_encoder_pos_matrix size
tensor([[[1., 1., 0., 0.],
         [1., 1., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]]) # 4*4，valid_encoder_pos_matrix 第一行表示第一个单词对其他单词的有效性
tensor([[[0., 0., 1., 1.],
         [0., 0., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]]) # invalid_encoder_pos_matrix 0表示有效位置，1表示无效的位置
tensor([[[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True,  True,  True],
         [ True,  True,  True,  True]],

        [[False, False, False, False],
         [False, False, False, False],
         [False, False, False, False],
         [False, False, False, False]]]) # mask_encoder_self_attention True的地方需要mask


In [6]:
# 用法，随机生成一个score
score = torch.randn(batch_size,max(src_len), max(src_len))
masked_score = score.masked_fill(mask_encoder_self_attention, -1e9) # 传入一个布尔型的张量，mask的地方置为负无穷
# 再对masked的score计算一个softmax, 计算出注意力的权重
prob = F.softmax(masked_score, -1)
print(prob, "# 注意力权重")

tensor([[[0.4961, 0.5039, 0.0000, 0.0000],
         [0.9147, 0.0853, 0.0000, 0.0000],
         [0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500]],

        [[0.2296, 0.1781, 0.2676, 0.3246],
         [0.5732, 0.1509, 0.0409, 0.2349],
         [0.1378, 0.3817, 0.0573, 0.4232],
         [0.0592, 0.6797, 0.0724, 0.1887]]]) # 注意力权重


In [22]:
# intra-attention mask实现
# 公式 Q valid pos @ K^T valid pos, shape:[batch_size, tgt_seq_len, src_seq_le]
# 构造tgt mask
valid_decoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L), (0, max(tgt_len)-L)), 0)for L in tgt_len]), 2)
print(valid_decoder_pos, "# valid_decoder_pos")
print(valid_decoder_pos.shape, "# valid_decoder_pos size")
# 构造出交叉有效位置 -- mask矩阵
valid_cross_pos_matrix = torch.bmm(valid_decoder_pos, valid_encoder_pos.transpose(1, 2))
print(valid_cross_pos_matrix, "# valid_cross_pos 目标序列对源序列关联有效矩阵，1表示有效, src len:[2,4] tgt len:[4,3], 所以一个tensor表示tgt 4个单词注意力到src的2个单词")
# 下面其实不用置反操作也行，使用masked_fill时候，参数mask == 0
invalid_cross_pos_matrix = 1-valid_cross_pos_matrix
print(invalid_cross_pos_matrix, "# invalid_cross_pos_matrix，1表示无效")
# 转换为bool类型
cross_attention_mask = invalid_cross_pos_matrix.to(torch.bool)
print(cross_attention_mask, "# cross_attention_mask， True表示需要mask的地方")

tensor([[[1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [0.]]]) # valid_decoder_pos
torch.Size([2, 4, 1]) # valid_decoder_pos size
tensor([[[1., 1., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 0., 0.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [0., 0., 0., 0.]]]) # valid_cross_pos 目标序列对源序列关联有效矩阵，1表示有效, src len:[2,4] tgt len:[4,3], 所以一个tensor表示tgt 4个单词注意力到src的2个单词
tensor([[[0., 0., 1., 1.],
         [0., 0., 1., 1.],
         [0., 0., 1., 1.],
         [0., 0., 1., 1.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [1., 1., 1., 1.]]]) # invalid_cross_pos_matrix，1表示无效
tensor([[[False, False,  True,  True],
         [False, False,  True,  True],
         [False, False,  True,  True],
         [False, False,  True,  True]],

        [[False, False, False, False],
         [False, False, False, False]

In [51]:
# 构造decoder self attention mask
# 构建一个下三角矩阵
valid_decoder_tri_matrix = [torch.tril(torch.ones(L,L)) for L in tgt_len] # tgt_len:[4, 3]
# pad操作是从低维到高维的，向后填充和向下填充
valid_decoder_tri_matrix = list(map(lambda x: F.pad(x, (0,max(tgt_len) - len(x),0,max(tgt_len) - len(x))), valid_decoder_tri_matrix))
print(valid_decoder_tri_matrix, "# valid_decoder_tri_matrix， 第二张量也变成四行了")
valid_decoder_tri_matrix = list(map(lambda x: torch.unsqueeze(x, 0), valid_decoder_tri_matrix))
valid_decoder_tri_matrix = torch.cat(valid_decoder_tri_matrix)
print(valid_decoder_tri_matrix.shape, "# valid_decoder_tri_matrix,扩维 拼接")
print("已经构建好了decoder self attention mask了")
invalid_decoder_tri_matrix = 1-valid_decoder_tri_matrix
invalid_decoder_tri_matrix = invalid_decoder_tri_matrix.to(torch.bool)
print(invalid_decoder_tri_matrix)

[tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]]), tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [0., 0., 0., 0.]])] # valid_decoder_tri_matrix， 第二张量也变成四行了
torch.Size([2, 4, 4]) # valid_decoder_tri_matrix,扩维 拼接
其实已经构建好了decoder self attention mask了
tensor([[[False,  True,  True,  True],
         [False, False,  True,  True],
         [False, False, False,  True],
         [False, False, False, False]],

        [[False,  True,  True,  True],
         [False, False,  True,  True],
         [False, False, False,  True],
         [ True,  True,  True,  True]]])


In [80]:
# 测试decoder self attention mask
score: torch.Tensor = torch.randn(batch_size, max(tgt_len), max(tgt_len))
score.masked_fill_(invalid_decoder_tri_matrix, -1e9)
print(score)
prob = F.softmax(score, dim=-1)
print(prob)

tensor([[[ 1.8276e+00, -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [ 2.8688e-01,  5.9017e-01, -1.0000e+09, -1.0000e+09],
         [ 8.5252e-01, -2.1590e+00, -6.6107e-01, -1.0000e+09],
         [-1.4719e+00, -6.4130e-01, -1.3947e+00, -1.7222e+00]],

        [[-3.9795e-01, -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [-5.3399e-01,  1.4487e+00, -1.0000e+09, -1.0000e+09],
         [-1.0009e+00,  2.0542e+00,  7.6253e-01, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09]]])
tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.4248, 0.5752, 0.0000, 0.0000],
         [0.7878, 0.0388, 0.1734, 0.0000],
         [0.1940, 0.4453, 0.2096, 0.1511]],

        [[1.0000, 0.0000, 0.0000, 0.0000],
         [0.1210, 0.8790, 0.0000, 0.0000],
         [0.0356, 0.7565, 0.2079, 0.0000],
         [0.2500, 0.2500, 0.2500, 0.2500]]])
